In [1]:
cd ~/Desktop/ML_project2/Tools

/Users/BotezatuCristian/Desktop/ML_project2/Tools


In [2]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
sns.set(style="whitegrid", palette="muted")
current_palette = sns.color_palette()
from scipy.io import loadmat
import scipy.linalg as linalg
import numpy as np
from scipy.stats import norm
from sklearn import tree
import graphviz
import sklearn.linear_model as lm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn import model_selection
from scipy import stats
import sys
sys.path.insert(0,'..\\02450Toolbox_Python\\Tools')
sys.path.insert(0,'..\\02450Toolbox_Python\\Scripts')
import similarity
import writeapriorifile

In [3]:
cd ~/Desktop/ML_project2

/Users/BotezatuCristian/Desktop/ML_project2


In [4]:
df = pd.read_excel('DataSet.xlsx')
sub_df = df[['hum', 'temp', 'windspeed', 'atemp','Date']]
sub_df = pd.concat([ df['new'].astype(np.float64), sub_df], axis=1)
sub_df = pd.concat([ df['registered'].astype(np.float64), sub_df], axis=1)
sub_df = pd.concat([ df['total'].astype(np.float64), sub_df], axis=1)
for i in ('new', 'registered', 'total'):
    sub_df[i] = (((sub_df[i] - sub_df[i].mean()) / sub_df[i].std()).abs())

In [5]:
countryNames = list(sub_df['Date'])
l = list(sub_df.columns)
attributesToShow = l[0:-1]
N = len(df)
X = sub_df.as_matrix(columns = attributesToShow)
std = np.std(X, axis=0)
std = np.expand_dims(std,axis=0)

X = X/std
# Subtract mean value from data
X = X - np.ones((len(X),1))*X.mean(0)
X

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


array([[ 1.73607061,  1.90353016, -0.00813447, ..., -0.4573892 ,
        -0.29378273, -0.36843775],
       [ 1.90093654,  1.88601082,  0.43956952, ..., -0.44135622,
         0.04102407, -0.38595851],
       [ 1.4099228 ,  1.27392895,  0.46419324, ..., -0.58010279,
         0.04014993, -0.67720433],
       ...,
       [ 1.41709088,  1.32539201,  0.37689096, ..., -0.53280422,
        -0.43084399,  1.97045458],
       [ 1.00940611,  1.05165235, -0.08200563, ..., -0.53072859,
         0.42950304,  1.83876435],
       [ 0.17342833,  0.11217786, -0.24989463, ..., -0.56393864,
        -0.31506613, -0.61680149]])

In [6]:
X_bin,attributesToShow_bin = similarity.binarize2(X,attributesToShow)

In [7]:
attributesToShow_bin

['total 50th-100th percentile',
 'registered 50th-100th percentile',
 'new 50th-100th percentile',
 'hum 50th-100th percentile',
 'temp 50th-100th percentile',
 'windspeed 50th-100th percentile',
 'atemp 50th-100th percentile',
 'total 0th-50th percentile',
 'registered 0th-50th percentile',
 'new 0th-50th percentile',
 'hum 0th-50th percentile',
 'temp 0th-50th percentile',
 'windspeed 0th-50th percentile',
 'atemp 0th-50th percentile']

# Association Mining

In [12]:
from apyori import apriori
import numpy as np
labels = ["total", "registered", "new", "hum", "temp", "windspeed", "atemp"]
def mat2transactions(X, labels=[]):
    T = []
    for i in range(X.shape[0]):
        l = np.nonzero(X[i, :])[0].tolist()
        if labels:
            l = [labels[i] for i in l]
        T.append(l)
    return T

# apyori requires data to be in a transactions format, forunately we just wrote a helper function to do that.
T = mat2transactions(X,labels)
rules = apriori( T, min_support=0.3, min_confidence=0.6)

def print_apriori_rules(rules):
    frules = []
    for r in rules:
        conf = r.ordered_statistics[0].confidence
        supp = r.support
        x = ", ".join( list( r.ordered_statistics[0].items_base ) )
        y = ", ".join( list( r.ordered_statistics[0].items_add ) )
        print("{%s} -> {%s}  (supp: %.3f, conf: %.3f)"%(x,y, supp, conf))
        frules.append( (x,y) )
    return frules
# Print rules found in the courses file.
print_apriori_rules(rules)

{atemp} -> {temp}  (supp: 0.416, conf: 0.833)
{new} -> {total}  (supp: 0.304, conf: 0.608)
{registered} -> {total}  (supp: 0.427, conf: 0.855)


[('atemp', 'temp'), ('new', 'total'), ('registered', 'total')]

In [13]:
T = mat2transactions(X_bin,labels=attributesToShow_bin)
rules = apriori(T, min_support=0.3, min_confidence=0.6)
print_apriori_rules(rules)

{atemp 0th-50th percentile} -> {new 50th-100th percentile}  (supp: 0.319, conf: 0.637)
{atemp 0th-50th percentile} -> {temp 0th-50th percentile}  (supp: 0.419, conf: 0.836)
{atemp 50th-100th percentile} -> {new 0th-50th percentile}  (supp: 0.319, conf: 0.638)
{atemp 50th-100th percentile} -> {temp 50th-100th percentile}  (supp: 0.416, conf: 0.833)
{new 0th-50th percentile} -> {temp 50th-100th percentile}  (supp: 0.305, conf: 0.609)
{new 0th-50th percentile} -> {total 0th-50th percentile}  (supp: 0.305, conf: 0.609)
{new 50th-100th percentile} -> {temp 0th-50th percentile}  (supp: 0.306, conf: 0.614)
{new 50th-100th percentile} -> {total 50th-100th percentile}  (supp: 0.304, conf: 0.608)
{registered 0th-50th percentile} -> {total 0th-50th percentile}  (supp: 0.428, conf: 0.855)
{registered 50th-100th percentile} -> {total 50th-100th percentile}  (supp: 0.427, conf: 0.855)


[('atemp 0th-50th percentile', 'new 50th-100th percentile'),
 ('atemp 0th-50th percentile', 'temp 0th-50th percentile'),
 ('atemp 50th-100th percentile', 'new 0th-50th percentile'),
 ('atemp 50th-100th percentile', 'temp 50th-100th percentile'),
 ('new 0th-50th percentile', 'temp 50th-100th percentile'),
 ('new 0th-50th percentile', 'total 0th-50th percentile'),
 ('new 50th-100th percentile', 'temp 0th-50th percentile'),
 ('new 50th-100th percentile', 'total 50th-100th percentile'),
 ('registered 0th-50th percentile', 'total 0th-50th percentile'),
 ('registered 50th-100th percentile', 'total 50th-100th percentile')]